# Transfer Learning

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import GlobalAveragePooling2D, Dense, Dropout
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from keras.preprocessing import image
from efficientnet.keras import EfficientNetB7
import os
import re

from keras.backend import clear_session
clear_session()

In [ ]:
base_path = ''
train_folders_path = os.path.join(base_path,'train')
validation_folders_path = ''
test_folders_path = os.path.join(base_path,'test')

In [ ]:
classlabels = os.listdir(train_folders_path)
print(classlabels)

In [ ]:
# Image dimensions
img_width, img_height = 224,224

## Importing and compiling the model

In [ ]:
ENB7Model = EfficientNetB7(include_top=False, input_shape= (img_width,img_height,3))
ENB7Model.trainable = False       #Freeze the weights of the transferred model
ENB7Model.summary()

In [ ]:
PredictionLayer = Dense(units = len(classlabels), activation = 'softmax')
model = Sequential([ENB7Model,   
                    Dropout(0.3),                           # Prevent overfitting by randomly dropping units with a probability of 30%
                    GlobalAveragePooling2D(),
                    PredictionLayer])


model.compile(optimizer = keras.optimizers.Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

## Generate the train and validation sets

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   rotation_range=30,
                                   zoom_range = 0.2,
                                   horizontal_flip = True,
                                   validation_split=0.2)

print('Making training data generator...')
training_set = datagen.flow_from_directory(train_folders_path,
                                           target_size = (img_width, img_height),
                                           batch_size = 32,
                                           class_mode = 'categorical',
                                           subset='training')

print('Making validation data generator...')
cv_set = datagen.flow_from_directory(train_folders_path,
                                       target_size = (img_width, img_height),
                                       batch_size = 32,
                                       class_mode = 'categorical',
                                       subset='validation',
                                       shuffle = False)

In [ ]:
ES1 = EarlyStopping(monitor='val_accuracy', patience=20)
MC1 = ModelCheckpoint(
    filepath=(os.path.join(base_path, 'EfficientNetB7 with top layers warmed.h5')),
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
    verbose=1)
LR1 = ReduceLROnPlateau(monitor='val_accuracy', factor=0.2, patience=5, min_lr = 1e-8, mode='max',verbose=1)

In [ ]:
print('Warming up the top layers')
history = model.fit_generator(training_set,
                         steps_per_epoch = (training_set.n // 32)+1,
                         epochs = 100,
                         validation_data = cv_set,
                         validation_steps = (cv_set.n//32)+1,
                         callbacks=[ES1,MC1,LR1])

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

## Fine Tuning

In [ ]:
ES2 = EarlyStopping(monitor='val_accuracy', patience=20)
MC2 = ModelCheckpoint(
    filepath=(os.path.join(base_path, 'EfficientNetB7 Full.h5')),
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
    verbose=1)
LR2 = ReduceLROnPlateau(monitor='val_accuracy', factor=0.2, patience=5, min_lr = 1e-8, mode='max',verbose=1)

In [ ]:
ENB7Model.trainable = True            # Unfreeze the pre trained weights
#for layer in ENB7Model.layers[:x]:    where x is an integer representing the layer you want to freeze it up till. ignore if you want to fine tune the whole model
#    layer.trainable = False

model.compile(optimizer = keras.optimizers.Adam(learning_rate=0.0001),    # slower learning rate to not update the pre trained weights too much
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

In [ ]:
history2 = model.fit_generator(training_set,
                         steps_per_epoch = (training_set.n // 32)+1,
                         epochs = 200,
                         initial_epoch =  history.epoch[-1],
                         validation_data = cv_set,
                         validation_steps = (cv_set.n//32)+1,
                         callbacks=[ES2,MC2,LR2])

In [ ]:
acc += history2.history['accuracy']
val_acc += history2.history['val_accuracy']

loss += history2.history['loss']
val_loss += history2.history['val_loss']

In [ ]:
plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

## For predicting with a csv format

In [ ]:
test = pd.read_csv('')
test.head()

In [ ]:
testimages_list=[]
for testimages_name in test['input column name here']:
    testimages_path = os.path.join(test_folders_path,testimages_name)
    testimages_list.append(testimages_path)
testimages_list

In [ ]:
from tqdm import tqdm
testresult_list = []
for img in tqdm(range(len(testimages_list))):
    test_image = image.load_img(testimages_list[img],target_size = (img_width, img_height))
    test_image = image.img_to_array(test_image)
    test_image = test_image / 255.
    test_image = np.expand_dims(test_image,axis=0)
    result = model.predict(test_image)
    highest_probability = int(np.argmax(result[0]))
    predicted_class = classlabels[highest_probability]
    test_result = [testimages_list[img],predicted_class]
    testresult_list.append(test_result)

In [ ]:
predicted_df = pd.DataFrame(testresult_list,columns=['Image','category'])
test['category']=predicted_df['category']
test['category']=test.category.apply(lambda c:str(c).zfill(2))

In [ ]:
test.to_csv(os.path.join(base_path,'test_final.csv'),index=False)